<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/11_superposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toy Models of Superposition
https://transformer-circuits.pub/2022/toy_model/

* このWebページのSection 2までを今回扱う。

## ニューラルネットワークによって得られる特徴量
* ニューラルネットワーク(NN) は、データの特徴量を自動的に抽出してくれると考えられている。
  * 手動での特徴量エンジニアリングを不要にしてくれたことが、NNの大きな貢献でもある。
* 例えば、多層パーセプトロンの特定の層に含まれる複数のニューロンを考える。
* これらのニューロンは、それぞれが別々の特徴量（例: 赤い色、左向きの曲線、犬の鼻、等）に反応すると考えていいのだろうか？
* そうとは限らないのでは？という考え方が、ある2022年の論文で提示されている。（上掲リンク先）
  * 多層パーセプトロンの特定の層のサイズを$m$とする。
  * サンプルをNNに入力すると、この層のニューロンの出力として$m$次元ベクトルが得られる。
  * この$m$次元ベクトルは、より高次元の$n$次元空間（$n \gg m$）の空間の直交基底を使って初めて表現できるような情報を含みうるということが、上掲論文では主張されている。
  * より高次元の空間でないと表現できない情報が、重ね合わされて（superposeされて）隠れベクトルにおいて表現されている、と主張されているのである。
  * つまり、NNは、場合によっては、ニューロンの個数よりも多い特徴量を表現していることがありうる、ということ。

## 以下のコードで何をしようとしているか
* 今回は、低い次元の空間のベクトルの集合が、その次元よりも高次元の情報を表現しうる（representしうる）ことを確認する。
  * 非常に簡単なモデルと、合成データとを使った、簡単な数値実験によって、このことを確認する。

### 数値実験の概要
* 例えば5次元のランダムなベクトルを、たくさん合成する。
  * ただし、これらはスパースなベクトルとする。
  * つまり、まずは一様乱数でベクトルの中身を埋めたあと、一定の確率でそれらを0にする。
  * これは、NNに入力される様々なサンプルが、限られた個数の特徴量しか持ち合わせていないことに対応している。
  * もちろん、あらゆるサンプルにわたって、そこに現れる特徴量の種類を調べると、その総数は非常に多いはずである。
  * しかし、個々のサンプルで見れば、わずかな数の特徴量しかそこには現れていないはずである。
* これら多数の5次元ベクトルを、特殊な方法で、例えば2次元に次元圧縮する。
  * すると、元の5次元ベクトルのsparsityが非常に高い場合、2次元空間に2個より多い「基底のようなもの」が計算されてくる。
  * そして、次元圧縮された2次元ベクトルは、これら、2個より多い「基底のようなもの」で表現されると考えることができる。

### 根拠: Johnson-Lindenstrauss lemma
* $N$個の$n$次元ベクトルの集合$X$を考える。
* $n$次元ベクトルを、より低い$m$次元のベクトルへと変換する線形写像は、いろいろありうるが・・・
  * $m \times n$の行列を持ってくればいいだけ。
* 実は、元の$n$次元空間でのベクトル間のユークリッド距離を"ほとんど変えない"線形写像が存在する。
* しかも、$m$は$\log(N)$のオーダまで小さくできる。
  * このオーダよりも小さい次元にすることはできない。
  * 元の空間の次元$n$ではなく、ベクトルの個数$N$に依存する量であることに注意。


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

%config InlineBackend.figure_format = 'retina'

import torch
import torch.nn as nn
import torch.optim as optim

np.random.seed(0)
torch.manual_seed(0)

### 高次元の特徴量の低次元表現を求めるためのモデル

In [ ]:
class SuperpositionModel(nn.Module):
    def __init__(self, dim_n, dim_m):
        super(SuperpositionModel, self).__init__()
        self.dim_n = dim_n
        self.dim_m = dim_m
        self.W = nn.Parameter(torch.randn(dim_n, dim_m))
        self.b = nn.Parameter(torch.zeros(dim_n))

    def forward(self, x):
        h = torch.matmul(x, self.W)
        x_reconstructed = torch.relu(torch.matmul(h, self.W.t()) + self.b)
        return x_reconstructed, h

* 高次元（$n$次元）の特徴量がsparseだと・・・
  * 同時に発火する特徴量が、少ない個数に限られることになる。
  * すると、ニューロン出力の空間の次元$m$が$n$より小さくても・・・
  * 高次元特徴量の$m$次元表現によって、それを再構成するときに・・・
  * 再構成のやり方が何通りもある、ということはなくなる。
  * （直交基底であれば、再構成のやり方は一通りしかない。）

* ということは、合成データを非常にスパースなベクトルにしておくと・・・
  * 高次元の特徴量のいずれについても・・・
  * それが低次元空間においてちゃんと表現されてくる可能性が高まる。

In [ ]:
def training(model, optimizer, sparsity=0.0, num_epochs=100000, num_data=10000, batch_size=100):
    dim_n = model.dim_n
    loss_weights = torch.pow(0.7, torch.arange(dim_n).float())
    print(f'Dimension of input: {dim_n}')
    print(f'Loss weights: {loss_weights}')

    # 高次元の特徴量は少ない個数だけが同時にactiveになるようにする
    X = torch.rand(num_data, dim_n) # 一様乱数
    zero_mask = (torch.rand_like(X) >= sparsity).float()
    X = X * zero_mask

    print("Starting training...")
    for epoch in range(num_epochs):
        x = X[torch.randint(0, num_data, (batch_size,))]
        model.train()
        optimizer.zero_grad()
        x_reconstructed, h = model(x)
        loss = ((x_reconstructed - x) ** 2 * loss_weights).mean()
        loss.backward()
        optimizer.step()
        if epoch % 10000 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
def evaluate(model):
    with torch.no_grad():
        feat_norm = torch.norm(model.W, dim=1).to('cpu').numpy()
        print("Feature norms:", feat_norm)
        superposition = ((model.W @ model.W.t()) ** 2 * (1.0 - torch.eye(model.dim_n))).sum(dim=1)
        print("Superposition:", superposition)
        return feat_norm, superposition.to('cpu').numpy()

In [ ]:
def visualize(feature_norm, superposition):
    fig, ax = plt.subplots(1, 2, figsize=(10, 6), gridspec_kw={'width_ratios': [4, 0.2]})
    y_pos = range(len(feature_norm))
    colors = plt.cm.viridis(superposition)
    ax[0].barh(y_pos[::-1], feature_norm, color=colors)
    ax[0].set_yticks(y_pos)
    ax[0].set_yticklabels([f'Feature {i}' for i in range(len(feature_norm) - 1, -1, -1)])
    ax[0].set_xlabel('Feature Norm')
    ax[0].set_title('Feature Norms and Superposition')
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=1))
    sm.set_array([])
    cbar = ax[1].figure.colorbar(sm, cax=ax[1])
    cbar.set_label('Superposition')
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
dim_n = 10
dim_m = 2

model = SuperpositionModel(dim_n, dim_m)
optimizer = optim.Adam(model.parameters(), lr=0.001)
training(model, optimizer, sparsity=0.99, num_data=10000, batch_size=100)
feature_norm, superposition = evaluate(model)
visualize(feature_norm, superposition)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
wf = ax.imshow((model.W @ model.W.t()).detach().to('cpu').numpy(), aspect=1, cmap='viridis')
ax.set_title('Inner Product Matrix')
ax.set_xticks([])
ax.set_yticks([])

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="7%", pad="5%")
cb1 = fig.colorbar(wf, cax=cax)

ax_bias = divider.append_axes("right", 1.2, pad=0.5, sharey=ax)
bf = ax_bias.imshow(model.b.detach().to('cpu').numpy().reshape(-1,1), aspect=1, cmap='inferno')
ax_bias.set_title('Bias')
ax_bias.set_xticks([])
ax_bias.set_yticks([])

cax = divider.append_axes("right", size="7%", pad=0)
cb1 = fig.colorbar(bf, cax=cax)

plt.show()

In [ ]:
# plot the features in 2D space as arrows
W_cpu = model.W.detach().to('cpu').numpy()
origin = np.zeros((dim_n, 2))
plt.quiver(*origin.T, W_cpu[:, 0], W_cpu[:, 1], angles='xy', scale_units='xy', scale=1)
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Features in 2D Space')
plt.grid()
plt.gca().set_aspect('equal')
plt.show()